# Lesson 6 — *Forms & User Interaction in Flask*

**Goal:** Learn how buttons and forms on a web page can **talk to a Flask API**.
- Click a button → get a **random student** profile from the API.  
- Type an ID and press a button → get that **student by ID** (with **validation**!).



## 0) Quick Setup

In [ ]:
# If Flask is missing, try installing it. (Run once)
try:
    import flask  # noqa: F401
    print("✅ Flask is already installed!")
except Exception:
    import sys, subprocess
    print("Installing Flask...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "flask"])
    print("✅ Flask installed!")

## 1) Our Student Data (Python dict → looks like JSON)
Run this and see the data structure. We'll serve it from Flask.


In [2]:
# Feel free to change or add more students!
# 🌟 Feel free to change or add more students!
students = {
    1: {
        "id": 1,
        "name": "Fatimah",
        "age": 12,
        "city": "Dallas",
        "favorite_color": "blue",
        "hobbies": ["drawing", "coding"]
    },
    2: {
        "id": 2,
        "name": "Ziyad",
        "age": 10,
        "city": "London",
        "favorite_color": "red",
        "hobbies": ["football", "gaming"]
    },
    3: {
        "id": 3,
        "name": "Maryam",
        "age": 13,
        "city": "Plano",
        "favorite_color": "green",
        "hobbies": ["reading", "basketball"]
    }
}

print("Sample students:")
for s in students.values():
    print(s)

Sample students:
{'id': 1, 'name': 'Fatimah', 'age': 12, 'city': 'Dallas', 'favorite_color': 'blue', 'hobbies': ['drawing', 'coding']}
{'id': 2, 'name': 'Ziyad', 'age': 10, 'city': 'London', 'favorite_color': 'red', 'hobbies': ['football', 'gaming']}
{'id': 3, 'name': 'Maryam', 'age': 13, 'city': 'Plano', 'favorite_color': 'green', 'hobbies': ['reading', 'basketball']}


### 1.1 Challenge
- Add **another student** (ID 4).  
- Give them at least **2 hobbies**.  
- Change **one favorite_color**.


## 2) Make the Flask API
We'll write **`app.py`** with these endpoints:
- `GET /api/random-student` → returns a **random** student
- `GET /api/student/<id>` → returns that student **by ID** (404 if not found)
- `GET /api/students` → returns all students (for testing)

This cell writes the file for you.


In [ ]:
from flask import Flask, jsonify, make_response, request
import random

app = Flask(__name__)

# --- Student data ---
students = {} # Replace this with your own data created in the previous challenge

def allow_cors(resp):
    resp.headers["Access-Control-Allow-Origin"] = "*"
    return resp

@app.route("/")
def home():
    return "<h2>✅ API ready! Try <code>/api/random-student</code> or <code>/api/student/1</code></h2>"

@app.route("/api/students")
def all_students():
    resp = make_response(jsonify(list(students.values())))
    return allow_cors(resp)

@app.route("/api/random-student")
def random_student():
    choice = random.choice(list(students.values()))
    resp = make_response(jsonify(choice))
    return allow_cors(resp)

@app.route("/api/student/<int:student_id>")
def student_by_id(student_id):
    data = students.get(student_id)
    if not data:
        resp = make_response(jsonify({"error": "Student not found"}), 404)
        return allow_cors(resp)
    resp = make_response(jsonify(data))
    return allow_cors(resp)

# Optional: an id via query string (e.g., /api/student?id=2)
@app.route("/api/student")
def student_by_query():
    try:
        sid = int(request.args.get("id", "").strip())
    except Exception:
        resp = make_response(jsonify({"error": "Invalid id"}), 400)
        return allow_cors(resp)
    data = students.get(sid)
    if not data:
        resp = make_response(jsonify({"error": "Student not found"}), 404)
        return allow_cors(resp)
    return allow_cors(make_response(jsonify(data)))

if __name__ == "__main__":
    app.run(debug=True)

SyntaxError: invalid syntax (4008411207.py, line 7)

### 2.1 Challenge
- Add a new field to each student (e.g., `"school"`).  
- Add a new endpoint: `GET /api/count` → returns how many students exist.  
- Bonus: Add `"image"` URLs and show them in the UI (next step).


## 3) Make the UI with Buttons + Form (Validation)
This page has:
- A **button** that fetches a **random student**.  
- A **text box + button** to fetch **by ID**, with **validation**.

This cell writes **`students_ui.html`** for you.


In [1]:
%%html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <title>Students — Forms & Buttons</title>
  <style>
    :root { --pink: #e10098; --muted:#6b7280; }
    body { font-family: system-ui, Arial, sans-serif; margin: 0; padding: 24px;  }
    .wrap { max-width: 720px; margin: 0 auto; }
    h1 { margin: 0 0 12px; }
    .row { display: flex; gap: 12px; flex-wrap: wrap; align-items: center; }
    button, input { padding: 10px 12px; border-radius: 10px; border: 1px solid #e5e7eb; }
    button { cursor: pointer; }
    .error { color: #b91c1c; margin-top: 6px; }
    .card {
      margin-top: 16px; background: #fff; border: 1px solid #e5e7eb; border-radius: 16px;
      padding: 16px; box-shadow: 0 6px 18px rgba(0,0,0,0.06);
    }
    .muted { color: var(--muted); margin: 0 0 6px; }
    .tag { display: inline-block; padding: 4px 10px; border-radius: 999px; background: #eef2ff; color: var(--pink); margin-right: 8px; margin-bottom: 8px; }
    .dot { display:inline-block; width: 10px; height: 10px; border-radius: 50%; border: 1px solid #ddd; vertical-align: middle; margin-right: 6px;}
    img { max-width: 100%; border-radius: 12px; margin-top: 8px; display:none; }
    .ok { color: #065f46; }
  </style>
</head>
<body>
  <div class="wrap">
    <h1>🎛️ Students — Forms & Buttons</h1>

    <div class="row">
      <button id="btnRandom">🎲 Get Random Student</button>
    </div>

    <div style="height: 8px"></div>

    <div>
      <div class="row">
        <input id="idInput" placeholder="Enter student ID (e.g., 1)" inputmode="numeric" />
        <button id="btnById">🔎 Get by ID</button>
      </div>
      <div id="idError" class="error" style="display:none;"></div>
    </div>

    <div class="card" id="out" style="display:none;">
      <div class="row">
        <span class="dot" id="colorDot"></span>
        <h2 id="name">Name</h2>
      </div>
      <p class="muted" id="city">City: —</p>
      <p class="muted" id="age">Age: —</p>
      <h3>Hobbies</h3>
      <div id="hobbies"></div>
      <img id="photo" alt="student image" />
      <p id="status" class="muted"></p>
    </div>
  </div>

  <script>
    // Change to your deployed domain if needed (e.g., PythonAnywhere):
    const API = "https://aelbarbary.pythonanywhere.com";

    const out = document.getElementById('out');
    const nameEl = document.getElementById('name');
    const cityEl = document.getElementById('city');
    const ageEl = document.getElementById('age');
    const hobbiesEl = document.getElementById('hobbies');
    const dot = document.getElementById('colorDot');
    const photo = document.getElementById('photo');
    const statusEl = document.getElementById('status');

    function showStudent(s, msg="") {
      if (!s || s.error) {
        out.style.display = 'none';
        statusEl.className = 'error';
        statusEl.textContent = s && s.error ? s.error : 'Unknown error';
        return;
      }
      out.style.display = 'block';
      nameEl.textContent = s.name || 'Unknown';
      cityEl.textContent = 'City: ' + (s.city || '—');
      ageEl.textContent  = 'Age: '  + (s.age  ?? '—');

      hobbiesEl.innerHTML = '';
      (s.hobbies || []).forEach(h => {
        const span = document.createElement('span');
        span.className = 'tag';
        span.textContent = h;
        hobbiesEl.appendChild(span);
      });

      if (s.favorite_color) dot.style.background = s.favorite_color;

      if (s.image) {
        photo.src = s.image;
        photo.style.display = 'block';
      } else {
        photo.style.display = 'none';
      }

      statusEl.className = msg ? 'ok' : 'muted';
      statusEl.textContent = msg;
    }

    async function getRandom() {
      statusEl.textContent = "Loading random student...";
      const r = await fetch(API + "/api/random-student");
      const data = await r.json();
      showStudent(data, "✅ Random student loaded!");
    }

    function validateId(raw) {
      const errorBox = document.getElementById('idError');
      const idStr = (raw || '').trim();
      if (!idStr) {
        errorBox.textContent = "Please type an ID.";
        errorBox.style.display = 'block';
        return null;
      }
      if (!/^[0-9]+$/.test(idStr)) {
        errorBox.textContent = "ID must be a number (e.g., 1).";
        errorBox.style.display = 'block';
        return null;
      }
      const id = parseInt(idStr, 10);
      if (id <= 0) {
        errorBox.textContent = "ID must be greater than 0.";
        errorBox.style.display = 'block';
        return null;
      }
      // OK
      errorBox.style.display = 'none';
      return id;
    }

    async function getById() {
      const input = document.getElementById('idInput');
      const id = validateId(input.value);
      if (id === null) {
        out.style.display = 'none';
        return;
      }
      statusEl.textContent = "Looking up student #" + id + "...";
      const r = await fetch(API + "/api/student/" + id);
      if (r.status === 404) {
        const data = await r.json();
        showStudent(data); // shows error
        return;
      }
      const data = await r.json();
      showStudent(data, "✅ Found student #" + id + "!");
    }

    document.getElementById('btnRandom').addEventListener('click', getRandom);
    document.getElementById('btnById').addEventListener('click', getById);
  </script>
</body>
</html>

### 3.1 Challenge
- If ID is invalid, make the **input border turn red**.  
- **Disable** the "Get by ID" button until the input is valid.  
- Add a **loading spinner** or message while fetching.  
- Bonus: If the student has an `"image"` field, show it (already supported!).


## 4) Bonus — Classic HTML **Form** (No JavaScript)
This version uses a normal form submit with `GET`.

This cell writes **`classic_form.html`** for you.


In [ ]:
%%writefile classic_form.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <title>Classic Form → Student by ID</title>
  <style>
    body { font-family: system-ui, Arial, sans-serif; padding: 24px; background: #f7fafc; }
    input, button { padding: 10px; border-radius: 8px; border: 1px solid #e5e7eb; }
    .error { color: #b91c1c; }
    .card { margin-top: 16px; background: #fff; border: 1px solid #e5e7eb; border-radius: 16px;
            padding: 16px; box-shadow: 0 6px 18px rgba(0,0,0,0.06); }
  </style>
</head>
<body>
  <h2>Classic Form (GET)</h2>
  <form action="http://127.0.0.1:5000/api/student" method="get">
    <input name="id" placeholder="Enter student ID" required pattern="^[0-9]+$" />
    <button type="submit">Go</button>
  </form>

  <p class="card">Note: This returns **raw JSON** in the browser. Try it!</p>
</body>
</html>

### 4.1 Challenge
- Change the **pattern** to only allow IDs from 1 to 99.  
- Add a friendly note telling users which IDs are available.


## 5) How to Run
1. Run the cell that wrote `app.py`.  
2. In a terminal, run: `python app.py` (it starts at `http://127.0.0.1:5000`).  
3. Open **`students_ui.html`** in your browser.  
4. Click **“Get Random Student”** or try an **ID** (like `1`).

> On PythonAnywhere: deploy `app.py` and update `API` in the HTML to your domain.


## 📝 End-of-Class Quiz (5 quick questions)
1) **Predict:** What HTTP method did our form use: GET or POST?  
2) **Fix it:** Is this valid JSON? `{id: 3, "name": "Maryam"}` (If not, fix it.)  
3) **Fill the blank:** To return JSON in Flask we use `_____ (data)`.  
4) **True/False:** Client-side validation can catch *all* bad inputs.  
5) **Mini code:** Add an endpoint `GET /api/count` that returns `{"count": <number of students>}`.
